# ============================================
# DISPUTE & REVENUE LEAKAGE INTELLIGENCE ANALYSIS
# NYC Yellow Taxi Data (2022-2025)
# ============================================

In [1]:
# ============================================
# SECTION 0: IMPORTS & CONFIGURATION
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.spatial.distance import mahalanobis
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from glob import glob
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Color scheme for visualizations
COLORS = {
    'normal': '#2ecc71',
    'dispute': '#e74c3c',
    'no_charge': '#f39c12',
    'voided': '#9b59b6',
    'anomaly': '#e67e22',
    'primary': '#3498db'
}

print("✅ Libraries imported successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

✅ Libraries imported successfully
Analysis Date: 2025-11-02 15:04:32


In [2]:
# ============================================
# SECTION 1: DATA LOADING & PREPARATION
# ============================================

print("\n📁 LOADING DATA...")
print("=" * 80)

# Define paths
PROCESSED_DATA_PATH = '/Users/yash/Documents/Projects/NYC_Yellow_Taxi_Analytics/data/processed/'
ZONE_LOOKUP_PATH = '/Users/yash/Documents/Projects/NYC_Yellow_Taxi_Analytics/data/raw/taxi_zone_lookup.csv'

# Find all cleaned parquet files
all_files = glob(os.path.join(PROCESSED_DATA_PATH, '**/*.parquet'), recursive=True)
print(f"Found {len(all_files)} parquet files")

# Load data (adjust number of files based on memory constraints)
# For full analysis, load all files; for testing, use subset
df_list = []
files_to_load = sorted(all_files)[-12:]  # Last 12 months

for file in files_to_load:
    try:
        df_temp = pd.read_parquet(file)
        df_list.append(df_temp)
        print(f"✓ Loaded: {os.path.basename(file)} - {len(df_temp):,} records")
    except Exception as e:
        print(f"✗ Error loading {os.path.basename(file)}: {str(e)}")

df = pd.concat(df_list, ignore_index=True)

print(f"\n✅ Total records loaded: {len(df):,}")
print(f"   Date range: {df['tpep_pickup_datetime'].min()} to {df['tpep_pickup_datetime'].max()}")
print(f"   Memory usage: {df.memory_usage(deep=True).sum() / (1024**3):.2f} GB")

# Load zone lookup
zones = pd.read_csv(ZONE_LOOKUP_PATH)
print(f"\n✓ Loaded {len(zones)} taxi zones")

# Display initial data info
print("\n📊 Dataset Overview:")
print(df.info())
print("\n📊 Sample Data:")
print(df.head())


📁 LOADING DATA...
Found 44 parquet files
✓ Loaded: yellow_tripdata_2024-09.parquet - 3,022,906 records
✓ Loaded: yellow_tripdata_2024-10.parquet - 3,300,536 records
✓ Loaded: yellow_tripdata_2024-11.parquet - 3,144,873 records
✓ Loaded: yellow_tripdata_2024-12.parquet - 3,198,189 records
✓ Loaded: yellow_tripdata_2025-01.parquet - 2,814,421 records
✓ Loaded: yellow_tripdata_2025-02.parquet - 2,655,706 records
✓ Loaded: yellow_tripdata_2025-03.parquet - 3,078,157 records
✓ Loaded: yellow_tripdata_2025-04.parquet - 3,056,792 records
✓ Loaded: yellow_tripdata_2025-05.parquet - 3,190,706 records
✓ Loaded: yellow_tripdata_2025-06.parquet - 2,909,840 records
✓ Loaded: yellow_tripdata_2025-07.parquet - 2,673,163 records
✓ Loaded: yellow_tripdata_2025-08.parquet - 2,510,535 records

✅ Total records loaded: 35,555,824
   Date range: 2002-12-31 22:17:43 to 2025-09-01 00:00:29
   Memory usage: 6.29 GB

✓ Loaded 265 taxi zones

📊 Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex:

In [3]:
# ============================================
# SECTION 1.1: DATA CLEANING & FEATURE ENGINEERING
# ============================================

print("\n🔧 DATA CLEANING & FEATURE ENGINEERING")
print("=" * 80)

# Store original count
original_count = len(df)

# 1. CREATE TIME-BASED FEATURES
print("\n1️⃣ Creating time-based features...")

df['trip_duration_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['pickup_day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
df['is_weekend'] = df['pickup_day_of_week'].isin([5, 6]).astype(int)
df['month_year'] = df['tpep_pickup_datetime'].dt.to_period('M')
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date

# Define rush hour (7-9 AM or 5-7 PM on weekdays)
df['is_rush_hour'] = (
    (~df['is_weekend'].astype(bool)) & 
    (((df['pickup_hour'] >= 7) & (df['pickup_hour'] <= 9)) | 
     ((df['pickup_hour'] >= 17) & (df['pickup_hour'] <= 19)))
).astype(int)

print(f"   ✓ Time features created")

# 2. CREATE FARE-BASED FEATURES
print("\n2️⃣ Creating fare-based features...")

# Handle division by zero
df['fare_per_mile'] = np.where(
    df['trip_distance'] > 0,
    df['fare_amount'] / df['trip_distance'],
    np.nan
)

df['fare_per_minute'] = np.where(
    df['trip_duration_minutes'] > 0,
    df['fare_amount'] / df['trip_duration_minutes'],
    np.nan
)

# Speed calculation
df['speed_mph'] = np.where(
    df['trip_duration_minutes'] > 0,
    (df['trip_distance'] / df['trip_duration_minutes']) * 60,
    np.nan
)

print(f"   ✓ Fare features created")

# 3. CREATE LOCATION FEATURES
print("\n3️⃣ Creating location features...")

# Merge with zone data
df = df.merge(
    zones[['LocationID', 'Borough', 'Zone']],
    left_on='PULocationID',
    right_on='LocationID',
    how='left'
).rename(columns={'Borough': 'pickup_borough', 'Zone': 'pickup_zone'})

df = df.merge(
    zones[['LocationID', 'Borough', 'Zone']],
    left_on='DOLocationID',
    right_on='LocationID',
    how='left',
    suffixes=('_pickup', '_dropoff')
).rename(columns={'Borough': 'dropoff_borough', 'Zone': 'dropoff_zone'})

df = df.drop(columns=['LocationID_pickup', 'LocationID_dropoff'], errors='ignore')

# Airport flag
airport_zones = ['JFK Airport', 'LaGuardia Airport', 'Newark Airport']
df['is_airport'] = (
    df['pickup_zone'].isin(airport_zones) | 
    df['dropoff_zone'].isin(airport_zones)
).astype(int)

print(f"   ✓ Location features created")

# 4. DATA QUALITY CLEANING
print("\n4️⃣ Cleaning data quality issues...")

# Remove obvious data quality issues
valid_mask = (
    (df['trip_distance'] >= 0) &
    (df['trip_distance'] <= 200) &  # Reasonable max distance
    (df['trip_duration_minutes'] > 0) &
    (df['trip_duration_minutes'] <= 300) &  # Max 5 hours
    (df['fare_amount'] >= 0) &
    (df['fare_amount'] <= 500) &  # Reasonable max fare
    (df['total_amount'] >= 0) &
    (df['passenger_count'] >= 0) &
    (df['passenger_count'] <= 8) &
    # Remove trips with zero distance but positive fare (unless payment issues)
    ~((df['trip_distance'] == 0) & (df['fare_amount'] > 5) & (~df['payment_type'].isin([3, 4, 6])))
)

df = df[valid_mask].copy()

removed_count = original_count - len(df)
print(f"   ✓ Removed {removed_count:,} invalid records ({removed_count/original_count*100:.2f}%)")
print(f"   ✓ Valid records: {len(df):,}")

# 5. HANDLE MISSING VALUES
print("\n5️⃣ Handling missing values...")

missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

if len(missing_summary) > 0:
    print("\n   Missing values found:")
    for col, count in missing_summary.items():
        print(f"      {col}: {count:,} ({count/len(df)*100:.2f}%)")
    
    # Fill missing values appropriately - check if columns exist first
    if 'pickup_borough' in df.columns:
        df['pickup_borough'] = df['pickup_borough'].fillna('Unknown')
    if 'dropoff_borough' in df.columns:
        df['dropoff_borough'] = df['dropoff_borough'].fillna('Unknown')
    if 'pickup_zone' in df.columns:
        df['pickup_zone'] = df['pickup_zone'].fillna('Unknown')
    if 'dropoff_zone' in df.columns:
        df['dropoff_zone'] = df['dropoff_zone'].fillna('Unknown')
    if 'congestion_surcharge' in df.columns:
        df['congestion_surcharge'] = df['congestion_surcharge'].fillna(0)
    if 'airport_fee' in df.columns:
        df['airport_fee'] = df['airport_fee'].fillna(0)
    else:
        # Create airport_fee column if it doesn't exist
        df['airport_fee'] = 0
        print("   ℹ️ airport_fee column not found - created with default value 0")
    
    # Handle other optional columns
    if 'cbd_congestion_fee' in df.columns:
        df['cbd_congestion_fee'] = df['cbd_congestion_fee'].fillna(0)
    else:
        df['cbd_congestion_fee'] = 0
        print("   ℹ️ cbd_congestion_fee column not found - created with default value 0")
    
    print("   ✓ Missing values handled")
else:
    print("   ✓ No missing values detected")
    # Still create columns if they don't exist
    if 'airport_fee' not in df.columns:
        df['airport_fee'] = 0
    if 'cbd_congestion_fee' not in df.columns:
        df['cbd_congestion_fee'] = 0

# 6. PAYMENT TYPE MAPPING
print("\n6️⃣ Creating payment type labels...")

payment_type_map = {
    0: 'Flex Fare',
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided'
}

df['payment_type_label'] = df['payment_type'].map(payment_type_map).fillna('Other')

print("\n📊 Payment Type Distribution:")
payment_dist = df['payment_type_label'].value_counts()
for payment_type, count in payment_dist.items():
    print(f"   {payment_type}: {count:,} ({count/len(df)*100:.2f}%)")

# 7. STORE-AND-FORWARD FLAG
print("\n7️⃣ Checking store-and-forward flag...")

if 'store_and_fwd_flag' in df.columns:
    saf_count = (df['store_and_fwd_flag'] == 'Y').sum()
    print(f"   ✓ Store-and-forward trips: {saf_count:,} ({saf_count/len(df)*100:.3f}%)")
else:
    df['store_and_fwd_flag'] = 'N'
    print("   ℹ️ store_and_fwd_flag not found - assuming all trips are normal")

print("\n✅ Data cleaning and feature engineering complete!")
print(f"   Final dataset: {len(df):,} records")
print(f"   Features: {len(df.columns)} columns")

# Display available columns for debugging
print("\n📋 Available columns:")
print(df.columns.tolist())


🔧 DATA CLEANING & FEATURE ENGINEERING

1️⃣ Creating time-based features...
   ✓ Time features created

2️⃣ Creating fare-based features...
   ✓ Fare features created

3️⃣ Creating location features...
   ✓ Location features created

4️⃣ Cleaning data quality issues...
   ✓ Removed 391 invalid records (0.00%)
   ✓ Valid records: 35,555,433

5️⃣ Handling missing values...

   Missing values found:
      cbd_congestion_fee: 12,666,397 (35.62%)
      dropoff_borough: 165,469 (0.47%)
      dropoff_zone: 105,382 (0.30%)
      pickup_zone: 83,978 (0.24%)
      pickup_borough: 8,445 (0.02%)
   ℹ️ airport_fee column not found - created with default value 0
   ✓ Missing values handled

6️⃣ Creating payment type labels...

📊 Payment Type Distribution:
   Credit Card: 30,286,040 (85.18%)
   Cash: 4,586,266 (12.90%)
   Dispute: 519,604 (1.46%)
   No Charge: 163,522 (0.46%)
   Unknown: 1 (0.00%)

7️⃣ Checking store-and-forward flag...
   ✓ Store-and-forward trips: 124,798 (0.351%)

✅ Data cleaning 

In [4]:
# ============================================
# SECTION 2: PROBLEM TRANSACTION IDENTIFICATION
# ============================================

print("\n🚨 IDENTIFYING PROBLEM TRANSACTIONS")
print("=" * 80)

# 1. PRIMARY PROBLEM FLAGS
print("\n1️⃣ Creating primary problem flags...")

df['is_dispute'] = (df['payment_type'] == 4).astype(int)
df['is_no_charge'] = (df['payment_type'] == 3).astype(int)
df['is_voided'] = (df['payment_type'] == 6).astype(int)
df['is_problematic'] = (df['is_dispute'] | df['is_no_charge'] | df['is_voided']).astype(int)

print(f"   ✓ Dispute trips: {df['is_dispute'].sum():,} ({df['is_dispute'].mean()*100:.3f}%)")
print(f"   ✓ No-charge trips: {df['is_no_charge'].sum():,} ({df['is_no_charge'].mean()*100:.3f}%)")
print(f"   ✓ Voided trips: {df['is_voided'].sum():,} ({df['is_voided'].mean()*100:.3f}%)")
print(f"   ✓ Total problematic: {df['is_problematic'].sum():,} ({df['is_problematic'].mean()*100:.3f}%)")

# 2. FARE ANOMALY DETECTION (IQR Method)
print("\n2️⃣ Detecting fare anomalies using IQR method...")

# Only consider trips with valid fare_per_mile
valid_fare_per_mile = df[df['fare_per_mile'].notna() & (df['fare_per_mile'] > 0)]['fare_per_mile']

Q1 = valid_fare_per_mile.quantile(0.25)
Q3 = valid_fare_per_mile.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df['is_fare_anomaly'] = (
    (df['fare_per_mile'] < lower_bound) | 
    (df['fare_per_mile'] > upper_bound)
).astype(int)

print(f"   ✓ Fare per mile range: ${lower_bound:.2f} - ${upper_bound:.2f}")
print(f"   ✓ Fare anomalies: {df['is_fare_anomaly'].sum():,} ({df['is_fare_anomaly'].mean()*100:.2f}%)")

# 3. DISTANCE ANOMALY DETECTION
print("\n3️⃣ Detecting distance anomalies...")

df['is_distance_anomaly'] = (
    ((df['trip_distance'] > 100) | 
     ((df['trip_distance'] < 0.1) & (df['fare_amount'] > 10)))
).astype(int)

print(f"   ✓ Distance anomalies: {df['is_distance_anomaly'].sum():,} ({df['is_distance_anomaly'].mean()*100:.2f}%)")

# 4. DURATION ANOMALY DETECTION
print("\n4️⃣ Detecting duration anomalies...")

df['is_duration_anomaly'] = (
    (df['trip_duration_minutes'] < 1) | 
    (df['trip_duration_minutes'] > 180)
).astype(int)

print(f"   ✓ Duration anomalies: {df['is_duration_anomaly'].sum():,} ({df['is_duration_anomaly'].mean()*100:.2f}%)")

# 5. SUSPICIOUS PATTERN DETECTION
print("\n5️⃣ Detecting suspicious patterns...")

df['is_suspicious'] = (
    # High distance with very low fare
    ((df['trip_distance'] > 20) & (df['fare_amount'] < 20)) |
    # Very short distance with high fare
    ((df['trip_distance'] < 0.5) & (df['fare_amount'] > 50)) |
    # High speed (likely data error or toll road)
    (df['speed_mph'] > 80) |
    # Extremely low speed (stuck in traffic or data error)
    ((df['speed_mph'] < 5) & (df['trip_distance'] > 5)) |
    # Zero fare but positive total (fees only)
    ((df['fare_amount'] == 0) & (df['total_amount'] > 0) & (~df['payment_type'].isin([3, 4, 6]))) |
    # Tip exceeds fare (generous or error)
    (df['tip_amount'] > df['fare_amount'] * 2)
).astype(int)

print(f"   ✓ Suspicious patterns: {df['is_suspicious'].sum():,} ({df['is_suspicious'].mean()*100:.2f}%)")

# 6. COMBINED ANOMALY FLAG
print("\n6️⃣ Creating combined anomaly flag...")

df['has_any_anomaly'] = (
    df['is_fare_anomaly'] | 
    df['is_distance_anomaly'] | 
    df['is_duration_anomaly'] | 
    df['is_suspicious']
).astype(int)

print(f"   ✓ Trips with any anomaly: {df['has_any_anomaly'].sum():,} ({df['has_any_anomaly'].mean()*100:.2f}%)")

# 7. SUMMARY STATISTICS
print("\n📊 PROBLEM TRANSACTION SUMMARY:")
print("=" * 80)

problem_summary = pd.DataFrame({
    'Category': ['Disputes', 'No Charge', 'Voided', 'Fare Anomaly', 
                 'Distance Anomaly', 'Duration Anomaly', 'Suspicious', 'Any Anomaly'],
    'Count': [
        df['is_dispute'].sum(),
        df['is_no_charge'].sum(),
        df['is_voided'].sum(),
        df['is_fare_anomaly'].sum(),
        df['is_distance_anomaly'].sum(),
        df['is_duration_anomaly'].sum(),
        df['is_suspicious'].sum(),
        df['has_any_anomaly'].sum()
    ],
    'Percentage': [
        df['is_dispute'].mean() * 100,
        df['is_no_charge'].mean() * 100,
        df['is_voided'].mean() * 100,
        df['is_fare_anomaly'].mean() * 100,
        df['is_distance_anomaly'].mean() * 100,
        df['is_duration_anomaly'].mean() * 100,
        df['is_suspicious'].mean() * 100,
        df['has_any_anomaly'].mean() * 100
    ]
})

print(problem_summary.to_string(index=False))

print("\n✅ Problem transaction identification complete!")


🚨 IDENTIFYING PROBLEM TRANSACTIONS

1️⃣ Creating primary problem flags...
   ✓ Dispute trips: 519,604 (1.461%)
   ✓ No-charge trips: 163,522 (0.460%)
   ✓ Voided trips: 0 (0.000%)
   ✓ Total problematic: 683,126 (1.921%)

2️⃣ Detecting fare anomalies using IQR method...
   ✓ Fare per mile range: $0.31 - $14.97
   ✓ Fare anomalies: 1,695,664 (4.77%)

3️⃣ Detecting distance anomalies...
   ✓ Distance anomalies: 48,802 (0.14%)

4️⃣ Detecting duration anomalies...
   ✓ Duration anomalies: 126,356 (0.36%)

5️⃣ Detecting suspicious patterns...
   ✓ Suspicious patterns: 81,263 (0.23%)

6️⃣ Creating combined anomaly flag...
   ✓ Trips with any anomaly: 1,725,463 (4.85%)

📊 PROBLEM TRANSACTION SUMMARY:
        Category   Count  Percentage
        Disputes  519604        1.46
       No Charge  163522        0.46
          Voided       0        0.00
    Fare Anomaly 1695664        4.77
Distance Anomaly   48802        0.14
Duration Anomaly  126356        0.36
      Suspicious   81263        0.23


In [ ]:
# ============================================
# SECTION 3: REVENUE LEAKAGE QUANTIFICATION
# ============================================

print("\n💰 REVENUE LEAKAGE QUANTIFICATION")
print("=" * 80)

# 1. DIRECT REVENUE LOSS
print("\n1️⃣ DIRECT REVENUE LOSS CALCULATION")
print("-" * 80)

# Calculate total revenue baseline
total_revenue = df['total_amount'].sum()
total_trips = len(df)

print(f"\n📊 Baseline Metrics:")
print(f"   Total trips: {total_trips:,}")
print(f"   Total revenue: ${total_revenue:,.2f}")
print(f"   Average fare: ${df['total_amount'].mean():.2f}")

# Direct losses by category
dispute_loss = df[df['is_dispute'] == 1]['total_amount'].sum()
no_charge_loss = df[df['is_no_charge'] == 1]['total_amount'].sum()
voided_loss = df[df['is_voided'] == 1]['total_amount'].sum()

dispute_count = df['is_dispute'].sum()
no_charge_count = df['is_no_charge'].sum()
voided_count = df['is_voided'].sum()

print(f"\n💸 DIRECT REVENUE LOSSES:")
print(f"\n   Disputes:")
print(f"      Trips: {dispute_count:,}")
print(f"      Revenue lost: ${dispute_loss:,.2f}")
print(f"      % of total revenue: {dispute_loss/total_revenue*100:.3f}%")
print(f"      Avg loss per trip: ${dispute_loss/max(dispute_count,1):.2f}")

print(f"\n   No Charge:")
print(f"      Trips: {no_charge_count:,}")
print(f"      Revenue lost: ${no_charge_loss:,.2f}")
print(f"      % of total revenue: {no_charge_loss/total_revenue*100:.3f}%")
print(f"      Avg loss per trip: ${no_charge_loss/max(no_charge_count,1):.2f}")

print(f"\n   Voided:")
print(f"      Trips: {voided_count:,}")
print(f"      Revenue lost: ${voided_loss:,.2f}")
print(f"      % of total revenue: {voided_loss/total_revenue*100:.3f}%")
print(f"      Avg loss per trip: ${voided_loss/max(voided_count,1):.2f}")

direct_loss_total = dispute_loss + no_charge_loss + voided_loss

print(f"\n   ⚠️ TOTAL DIRECT LOSS: ${direct_loss_total:,.2f} ({direct_loss_total/total_revenue*100:.3f}%)")

# 2. ANOMALY REVENUE LOSS
print("\n\n2️⃣ ANOMALY REVENUE LOSS CALCULATION")
print("-" * 80)

# Calculate expected fare based on median fare_per_mile
median_fare_per_mile = df[df['fare_per_mile'].notna() & (df['fare_per_mile'] > 0)]['fare_per_mile'].median()

print(f"\n📊 Benchmark: Median fare per mile = ${median_fare_per_mile:.2f}")

# For anomalous trips, calculate expected vs actual
anomaly_trips = df[df['is_fare_anomaly'] == 1].copy()
anomaly_trips['expected_fare'] = anomaly_trips['trip_distance'] * median_fare_per_mile
anomaly_trips['fare_difference'] = anomaly_trips['fare_amount'] - anomaly_trips['expected_fare']

# Separate into overcharges and undercharges
undercharges = anomaly_trips[anomaly_trips['fare_difference'] < 0]
overcharges = anomaly_trips[anomaly_trips['fare_difference'] > 0]

undercharge_loss = abs(undercharges['fare_difference'].sum())
overcharge_gain = overcharges['fare_difference'].sum()
net_anomaly_loss = undercharge_loss - overcharge_gain

print(f"\n💸 ANOMALY REVENUE IMPACT:")
print(f"\n   Undercharges:")
print(f"      Trips: {len(undercharges):,}")
print(f"      Revenue lost: ${undercharge_loss:,.2f}")
print(f"      Avg undercharge: ${undercharge_loss/max(len(undercharges),1):.2f}")

print(f"\n   Overcharges:")
print(f"      Trips: {len(overcharges):,}")
print(f"      Revenue gained: ${overcharge_gain:,.2f}")
print(f"      Avg overcharge: ${overcharge_gain/max(len(overcharges),1):.2f}")

print(f"\n   NET ANOMALY LOSS: ${net_anomaly_loss:,.2f}")

# 3. TOTAL REVENUE LEAKAGE
print("\n\n3️⃣ TOTAL REVENUE LEAKAGE SUMMARY")
print("=" * 80)

# Create comprehensive summary table
leakage_summary = pd.DataFrame({
    'Loss Category': [
        'Disputes',
        'No Charge',
        'Voided',
        'Fare Anomalies (Net)',
        '─' * 30,
        'TOTAL LEAKAGE'
    ],
    'Trip Count': [
        dispute_count,
        no_charge_count,
        voided_count,
        len(anomaly_trips),
        '─' * 10,
        dispute_count + no_charge_count + voided_count
    ],
    'Total $ Lost': [
        f'${dispute_loss:,.2f}',
        f'${no_charge_loss:,.2f}',
        f'${voided_loss:,.2f}',
        f'${net_anomaly_loss:,.2f}',
        '─' * 15,
        f'${direct_loss_total + net_anomaly_loss:,.2f}'
    ],
    '% of Total Revenue': [
        f'{dispute_loss/total_revenue*100:.3f}%',
        f'{no_charge_loss/total_revenue*100:.3f}%',
        f'{voided_loss/total_revenue*100:.3f}%',
        f'{net_anomaly_loss/total_revenue*100:.3f}%',
        '─' * 8,
        f'{(direct_loss_total + net_anomaly_loss)/total_revenue*100:.3f}%'
    ],
    'Avg Loss per Trip': [
        f'${dispute_loss/max(dispute_count,1):.2f}',
        f'${no_charge_loss/max(no_charge_count,1):.2f}',
        f'${voided_loss/max(voided_count,1):.2f}',
        f'${net_anomaly_loss/max(len(anomaly_trips),1):.2f}',
        '─' * 8,
        f'${(direct_loss_total + net_anomaly_loss)/max(dispute_count + no_charge_count + voided_count,1):.2f}'
    ]
})

print("\n" + leakage_summary.to_string(index=False))

# 4. ANNUALIZED PROJECTION
print("\n\n4️⃣ ANNUALIZED IMPACT PROJECTION")
print("-" * 80)

# Calculate date range
date_range_days = (df['pickup_date'].max() - df['pickup_date'].min()).days
years_covered = date_range_days / 365.25

total_leakage = direct_loss_total + net_anomaly_loss

if years_covered < 1:
    annual_leakage = (total_leakage / date_range_days) * 365.25
    print(f"\n   Data coverage: {date_range_days} days ({years_covered:.2f} years)")
    print(f"   Total leakage in period: ${total_leakage:,.2f}")
    print(f"   📈 PROJECTED ANNUAL LEAKAGE: ${annual_leakage:,.2f}")
else:
    annual_leakage = total_leakage / years_covered
    print(f"\n   Data coverage: {date_range_days} days ({years_covered:.2f} years)")
    print(f"   Total leakage in period: ${total_leakage:,.2f}")
    print(f"   📈 AVERAGE ANNUAL LEAKAGE: ${annual_leakage:,.2f}")

# Store key metrics for later use
leakage_metrics = {
    'total_revenue': total_revenue,
    'total_trips': total_trips,
    'dispute_loss': dispute_loss,
    'no_charge_loss': no_charge_loss,
    'voided_loss': voided_loss,
    'direct_loss_total': direct_loss_total,
    'net_anomaly_loss': net_anomaly_loss,
    'total_leakage': total_leakage,
    'annual_leakage': annual_leakage,
    'leakage_percentage': (total_leakage / total_revenue) * 100
}

print("\n✅ Revenue leakage quantification complete!")
print(f"\n🚨 KEY FINDING: ${total_leakage:,.2f} in total revenue leakage")
print(f"   This represents {leakage_metrics['leakage_percentage']:.3f}% of total revenue")


💰 REVENUE LEAKAGE QUANTIFICATION

1️⃣ DIRECT REVENUE LOSS CALCULATION
--------------------------------------------------------------------------------

📊 Baseline Metrics:
   Total trips: 35,555,433
   Total revenue: $1,039,869,575.43
   Average fare: $29.25

💸 DIRECT REVENUE LOSSES:

   Disputes:
      Trips: 519,604
      Revenue lost: $15,147,393.45
      % of total revenue: 1.457%
      Avg loss per trip: $29.15

   No Charge:
      Trips: 163,522
      Revenue lost: $4,227,031.42
      % of total revenue: 0.406%
      Avg loss per trip: $25.85

   Voided:
      Trips: 0
      Revenue lost: $0.00
      % of total revenue: 0.000%
      Avg loss per trip: $0.00

   ⚠️ TOTAL DIRECT LOSS: $19,374,424.87 (1.863%)


2️⃣ ANOMALY REVENUE LOSS CALCULATION
--------------------------------------------------------------------------------

📊 Benchmark: Median fare per mile = $7.41


In [ ]:
# ============================================
# SECTION 4: TEMPORAL PATTERN ANALYSIS
# ============================================

print("\n⏰ TEMPORAL PATTERN ANALYSIS")
print("=" * 80)

# 1. HOURLY PATTERNS
print("\n1️⃣ ANALYZING HOURLY PATTERNS")
print("-" * 80)

hourly_analysis = df.groupby('pickup_hour').agg({
    'is_dispute': ['sum', 'mean'],
    'is_no_charge': ['sum', 'mean'],
    'is_voided': ['sum', 'mean'],
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'mean', 'count']
}).reset_index()

hourly_analysis.columns = ['hour', 'dispute_count', 'dispute_rate', 
                           'no_charge_count', 'no_charge_rate',
                           'voided_count', 'voided_rate',
                           'problem_count', 'problem_rate',
                           'total_revenue', 'avg_fare', 'trip_count']

# Calculate revenue loss per hour
hourly_problem_revenue = df[df['is_problematic'] == 1].groupby('pickup_hour')['total_amount'].sum().reset_index()
hourly_problem_revenue.columns = ['hour', 'revenue_lost']
hourly_analysis = hourly_analysis.merge(hourly_problem_revenue, on='hour', how='left').fillna(0)

print("\n📊 Hourly Problem Rate Summary:")
print(hourly_analysis[['hour', 'trip_count', 'problem_rate', 'revenue_lost']].to_string(index=False))

# Identify peak problem hours
top_problem_hours = hourly_analysis.nlargest(5, 'problem_rate')[['hour', 'problem_rate', 'revenue_lost']]
print(f"\n⚠️ TOP 5 PROBLEM HOURS:")
print(top_problem_hours.to_string(index=False))

# 2. DAY OF WEEK PATTERNS
print("\n\n2️⃣ ANALYZING DAY OF WEEK PATTERNS")
print("-" * 80)

dow_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
           4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

dow_analysis = df.groupby('pickup_day_of_week').agg({
    'is_dispute': ['sum', 'mean'],
    'is_no_charge': ['sum', 'mean'],
    'is_voided': ['sum', 'mean'],
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'count']
}).reset_index()

dow_analysis.columns = ['day_of_week', 'dispute_count', 'dispute_rate',
                        'no_charge_count', 'no_charge_rate',
                        'voided_count', 'voided_rate',
                        'problem_count', 'problem_rate',
                        'total_revenue', 'trip_count']

dow_analysis['day_name'] = dow_analysis['day_of_week'].map(dow_map)

# Calculate revenue loss per day
dow_problem_revenue = df[df['is_problematic'] == 1].groupby('pickup_day_of_week')['total_amount'].sum().reset_index()
dow_problem_revenue.columns = ['day_of_week', 'revenue_lost']
dow_analysis = dow_analysis.merge(dow_problem_revenue, on='day_of_week', how='left').fillna(0)

print("\n📊 Day of Week Problem Rate Summary:")
print(dow_analysis[['day_name', 'trip_count', 'problem_rate', 'revenue_lost']].to_string(index=False))

# Weekend vs Weekday comparison
weekend_problem_rate = df[df['is_weekend'] == 1]['is_problematic'].mean()
weekday_problem_rate = df[df['is_weekend'] == 0]['is_problematic'].mean()

print(f"\n📅 WEEKEND vs WEEKDAY:")
print(f"   Weekend problem rate: {weekend_problem_rate*100:.3f}%")
print(f"   Weekday problem rate: {weekday_problem_rate*100:.3f}%")
print(f"   Difference: {(weekend_problem_rate - weekday_problem_rate)*100:+.3f} percentage points")

# 3. MONTHLY TRENDS
print("\n\n3️⃣ ANALYZING MONTHLY TRENDS")
print("-" * 80)

monthly_analysis = df.groupby('month_year').agg({
    'is_dispute': ['sum', 'mean'],
    'is_no_charge': ['sum', 'mean'],
    'is_voided': ['sum', 'mean'],
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'count']
}).reset_index()

monthly_analysis.columns = ['month', 'dispute_count', 'dispute_rate',
                            'no_charge_count', 'no_charge_rate',
                            'voided_count', 'voided_rate',
                            'problem_count', 'problem_rate',
                            'total_revenue', 'trip_count']

# Calculate revenue loss per month
monthly_problem_revenue = df[df['is_problematic'] == 1].groupby('month_year')['total_amount'].sum().reset_index()
monthly_problem_revenue.columns = ['month', 'revenue_lost']
monthly_analysis = monthly_analysis.merge(monthly_problem_revenue, on='month', how='left').fillna(0)

monthly_analysis['month_str'] = monthly_analysis['month'].astype(str)

print("\n📊 Monthly Trend Summary:")
print(monthly_analysis[['month_str', 'trip_count', 'problem_rate', 'revenue_lost']].to_string(index=False))

# Trend analysis
if len(monthly_analysis) > 1:
    first_month_rate = monthly_analysis.iloc[0]['problem_rate']
    last_month_rate = monthly_analysis.iloc[-1]['problem_rate']
    trend_change = last_month_rate - first_month_rate
    
    print(f"\n📈 TREND ANALYSIS:")
    print(f"   First month problem rate: {first_month_rate*100:.3f}%")
    print(f"   Last month problem rate: {last_month_rate*100:.3f}%")
    print(f"   Change: {trend_change*100:+.3f} percentage points")
    
    if trend_change > 0.001:
        print(f"   ⚠️ ALERT: Problem rates are INCREASING")
    elif trend_change < -0.001:
        print(f"   ✅ GOOD NEWS: Problem rates are DECREASING")
    else:
        print(f"   → Problem rates are STABLE")

# 4. RUSH HOUR ANALYSIS
print("\n\n4️⃣ RUSH HOUR vs OFF-PEAK COMPARISON")
print("-" * 80)

# FIX: Use separate aggregations to avoid MultiIndex confusion
rush_hour_df = df[df['is_rush_hour'] == 1]
off_peak_df = df[df['is_rush_hour'] == 0]

rush_hour_trips = len(rush_hour_df)
rush_hour_problem_count = rush_hour_df['is_problematic'].sum()
rush_hour_problem_rate = rush_hour_df['is_problematic'].mean()
rush_hour_revenue_lost = rush_hour_df[rush_hour_df['is_problematic'] == 1]['total_amount'].sum()

off_peak_trips = len(off_peak_df)
off_peak_problem_count = off_peak_df['is_problematic'].sum()
off_peak_problem_rate = off_peak_df['is_problematic'].mean()
off_peak_revenue_lost = off_peak_df[off_peak_df['is_problematic'] == 1]['total_amount'].sum()

print(f"\n📊 RUSH HOUR (7-9 AM, 5-7 PM Weekdays):")
print(f"   Trips: {rush_hour_trips:,}")
print(f"   Problem rate: {rush_hour_problem_rate*100:.3f}%")
print(f"   Problem trips: {rush_hour_problem_count:,}")
print(f"   Revenue lost: ${rush_hour_revenue_lost:,.2f}")

print(f"\n📊 OFF-PEAK:")
print(f"   Trips: {off_peak_trips:,}")
print(f"   Problem rate: {off_peak_problem_rate*100:.3f}%")
print(f"   Problem trips: {off_peak_problem_count:,}")
print(f"   Revenue lost: ${off_peak_revenue_lost:,.2f}")

print(f"\n   Difference: {(rush_hour_problem_rate - off_peak_problem_rate)*100:+.3f} percentage points")

if rush_hour_problem_rate > off_peak_problem_rate:
    print(f"   ⚠️ ALERT: Rush hour has {((rush_hour_problem_rate / off_peak_problem_rate) - 1) * 100:.1f}% MORE problems")
else:
    print(f"   ✅ Rush hour has {((1 - rush_hour_problem_rate / off_peak_problem_rate)) * 100:.1f}% FEWER problems")

print("\n✅ Temporal pattern analysis complete!")

# Store for visualizations
temporal_data = {
    'hourly': hourly_analysis,
    'dow': dow_analysis,
    'monthly': monthly_analysis
}

In [ ]:
# ============================================
# SECTION 5: GEOGRAPHIC HOTSPOT ANALYSIS
# ============================================

print("\n🗺️ GEOGRAPHIC HOTSPOT ANALYSIS")
print("=" * 80)

# 1. PICKUP ZONE ANALYSIS
print("\n1️⃣ ANALYZING PICKUP ZONES")
print("-" * 80)

pickup_analysis = df.groupby('PULocationID').agg({
    'is_dispute': ['sum', 'mean'],
    'is_no_charge': ['sum', 'mean'],
    'is_voided': ['sum', 'mean'],
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'count'],
    'is_fare_anomaly': 'mean'
}).reset_index()

pickup_analysis.columns = ['LocationID', 'dispute_count', 'dispute_rate',
                           'no_charge_count', 'no_charge_rate',
                           'voided_count', 'voided_rate',
                           'problem_count', 'problem_rate',
                           'total_revenue', 'trip_count',
                           'anomaly_score']

# Calculate revenue lost per zone
pickup_revenue_lost = df[df['is_problematic'] == 1].groupby('PULocationID')['total_amount'].sum().reset_index()
pickup_revenue_lost.columns = ['LocationID', 'revenue_lost']
pickup_analysis = pickup_analysis.merge(pickup_revenue_lost, on='LocationID', how='left').fillna(0)

# Merge with zone names
pickup_analysis = pickup_analysis.merge(
    zones[['LocationID', 'Borough', 'Zone']],
    on='LocationID',
    how='left'
)

# Calculate problem score (weighted combination of metrics)
pickup_analysis['problem_score'] = (
    pickup_analysis['problem_rate'] * 0.4 +
    pickup_analysis['anomaly_score'] * 0.3 +
    (pickup_analysis['revenue_lost'] / pickup_analysis['revenue_lost'].max()) * 0.3
)

# Sort by revenue lost
pickup_analysis = pickup_analysis.sort_values('revenue_lost', ascending=False)

print("\n📊 TOP 20 PROBLEM PICKUP ZONES (by Revenue Lost):")
print(pickup_analysis[['Zone', 'Borough', 'trip_count', 'problem_rate', 'revenue_lost']].head(20).to_string(index=False))

# 2. DROPOFF ZONE ANALYSIS
print("\n\n2️⃣ ANALYZING DROPOFF ZONES")
print("-" * 80)

dropoff_analysis = df.groupby('DOLocationID').agg({
    'is_dispute': ['sum', 'mean'],
    'is_no_charge': ['sum', 'mean'],
    'is_voided': ['sum', 'mean'],
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'count'],
    'is_fare_anomaly': 'mean'
}).reset_index()

dropoff_analysis.columns = ['LocationID', 'dispute_count', 'dispute_rate',
                            'no_charge_count', 'no_charge_rate',
                            'voided_count', 'voided_rate',
                            'problem_count', 'problem_rate',
                            'total_revenue', 'trip_count',
                            'anomaly_score']

# Calculate revenue lost per zone
dropoff_revenue_lost = df[df['is_problematic'] == 1].groupby('DOLocationID')['total_amount'].sum().reset_index()
dropoff_revenue_lost.columns = ['LocationID', 'revenue_lost']
dropoff_analysis = dropoff_analysis.merge(dropoff_revenue_lost, on='LocationID', how='left').fillna(0)

# Merge with zone names
dropoff_analysis = dropoff_analysis.merge(
    zones[['LocationID', 'Borough', 'Zone']],
    on='LocationID',
    how='left'
)

dropoff_analysis = dropoff_analysis.sort_values('revenue_lost', ascending=False)

print("\n📊 TOP 20 PROBLEM DROPOFF ZONES (by Revenue Lost):")
print(dropoff_analysis[['Zone', 'Borough', 'trip_count', 'problem_rate', 'revenue_lost']].head(20).to_string(index=False))

# 3. ROUTE-LEVEL ANALYSIS
print("\n\n3️⃣ ANALYZING HIGH-RISK ROUTES")
print("-" * 80)

route_analysis = df.groupby(['PULocationID', 'DOLocationID']).agg({
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'count']
}).reset_index()

route_analysis.columns = ['PULocationID', 'DOLocationID', 
                         'problem_count', 'problem_rate',
                         'total_revenue', 'trip_count']

# Calculate revenue at risk
route_revenue_lost = df[df['is_problematic'] == 1].groupby(['PULocationID', 'DOLocationID'])['total_amount'].sum().reset_index()
route_revenue_lost.columns = ['PULocationID', 'DOLocationID', 'revenue_lost']
route_analysis = route_analysis.merge(route_revenue_lost, on=['PULocationID', 'DOLocationID'], how='left').fillna(0)

# Add zone names
route_analysis = route_analysis.merge(
    zones[['LocationID', 'Zone']].rename(columns={'LocationID': 'PULocationID', 'Zone': 'pickup_zone'}),
    on='PULocationID',
    how='left'
)
route_analysis = route_analysis.merge(
    zones[['LocationID', 'Zone']].rename(columns={'LocationID': 'DOLocationID', 'Zone': 'dropoff_zone'}),
    on='DOLocationID',
    how='left'
)

# Filter routes with significant volume
route_analysis = route_analysis[route_analysis['trip_count'] >= 100]
route_analysis = route_analysis.sort_values('revenue_lost', ascending=False)

print("\n📊 TOP 30 HIGH-RISK ROUTES:")
print(route_analysis[['pickup_zone', 'dropoff_zone', 'trip_count', 'problem_rate', 'revenue_lost']].head(30).to_string(index=False))

# 4. BOROUGH-LEVEL SUMMARY
print("\n\n4️⃣ BOROUGH-LEVEL SUMMARY")
print("-" * 80)

# Pickup borough analysis
pickup_borough = df.groupby('pickup_borough').agg({
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'count']
}).reset_index()

pickup_borough.columns = ['Borough', 'problem_count', 'problem_rate', 'total_revenue', 'trip_count']

pickup_borough_lost = df[df['is_problematic'] == 1].groupby('pickup_borough')['total_amount'].sum().reset_index()
pickup_borough_lost.columns = ['Borough', 'revenue_lost']
pickup_borough = pickup_borough.merge(pickup_borough_lost, on='Borough', how='left').fillna(0)

pickup_borough = pickup_borough.sort_values('revenue_lost', ascending=False)

print("\n📊 PICKUP BOROUGH ANALYSIS:")
print(pickup_borough.to_string(index=False))

print("\n✅ Geographic hotspot analysis complete!")

# Store for visualizations
geographic_data = {
    'pickup_zones': pickup_analysis,
    'dropoff_zones': dropoff_analysis,
    'routes': route_analysis,
    'boroughs': pickup_borough
}

In [ ]:
# ============================================
# SECTION 6: VENDOR ACCOUNTABILITY ANALYSIS
# ============================================

print("\n🏢 VENDOR ACCOUNTABILITY ANALYSIS")
print("=" * 80)

# 1. VENDOR PERFORMANCE METRICS
print("\n1️⃣ CALCULATING VENDOR PERFORMANCE METRICS")
print("-" * 80)

vendor_analysis = df.groupby('VendorID').agg({
    'is_dispute': ['sum', 'mean'],
    'is_no_charge': ['sum', 'mean'],
    'is_voided': ['sum', 'mean'],
    'is_problematic': ['sum', 'mean'],
    'total_amount': ['sum', 'mean', 'count'],
    'fare_per_mile': 'mean',
    'tip_amount': 'mean'
}).reset_index()

vendor_analysis.columns = ['VendorID', 'dispute_count', 'dispute_rate',
                           'no_charge_count', 'no_charge_rate',
                           'voided_count', 'voided_rate',
                           'problem_count', 'problem_rate',
                           'total_revenue', 'avg_fare', 'trip_count',
                           'avg_fare_per_mile', 'avg_tip']

# Calculate revenue lost per vendor
vendor_revenue_lost = df[df['is_problematic'] == 1].groupby('VendorID')['total_amount'].sum().reset_index()
vendor_revenue_lost.columns = ['VendorID', 'revenue_lost']
vendor_analysis = vendor_analysis.merge(vendor_revenue_lost, on='VendorID', how='left').fillna(0)

# Calculate store-and-forward rate
vendor_saf = df[df['store_and_fwd_flag'] == 'Y'].groupby('VendorID').size().reset_index(name='saf_count')
vendor_analysis = vendor_analysis.merge(vendor_saf, on='VendorID', how='left').fillna(0)
vendor_analysis['saf_rate'] = vendor_analysis['saf_count'] / vendor_analysis['trip_count']

# Vendor name mapping (if known)
vendor_names = {
    1: 'Creative Mobile Technologies',
    2: 'VeriFone Inc.'
}
vendor_analysis['VendorName'] = vendor_analysis['VendorID'].map(vendor_names).fillna('Unknown')

print("\n📊 VENDOR PERFORMANCE COMPARISON:")
print(vendor_analysis[['VendorName', 'trip_count', 'problem_rate', 'dispute_rate', 
                       'avg_fare', 'revenue_lost', 'saf_rate']].to_string(index=False))

# 2. STATISTICAL TESTING
print("\n\n2️⃣ STATISTICAL SIGNIFICANCE TESTING")
print("-" * 80)

if len(vendor_analysis) > 1:
    # Chi-square test for dispute rates
    from scipy.stats import chi2_contingency
    
    contingency_table = pd.crosstab(df['VendorID'], df['is_problematic'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\n📊 Chi-Square Test for Problem Rate Differences:")
    print(f"   Chi-square statistic: {chi2:.4f}")
    print(f"   P-value: {p_value:.6f}")
    print(f"   Degrees of freedom: {dof}")
    
    if p_value < 0.05:
        print(f"   ✅ SIGNIFICANT: Vendor problem rates differ significantly (p < 0.05)")
    else:
        print(f"   → NOT SIGNIFICANT: Vendor problem rates are similar (p >= 0.05)")
    
    # Calculate relative risk
    overall_problem_rate = df['is_problematic'].mean()
    
    print(f"\n📊 RELATIVE RISK ANALYSIS:")
    print(f"   Overall problem rate: {overall_problem_rate*100:.3f}%")
    
    for idx, row in vendor_analysis.iterrows():
        relative_risk = (row['problem_rate'] / overall_problem_rate - 1) * 100
        print(f"   {row['VendorName']}: {relative_risk:+.1f}% vs average")

# 3. VENDOR TECHNOLOGY ISSUES
print("\n\n3️⃣ TECHNOLOGY & CONNECTIVITY ANALYSIS")
print("-" * 80)

# Analyze correlation between store-and-forward and problems
saf_problem_corr = df[['store_and_fwd_flag', 'is_problematic']].copy()
saf_problem_corr['saf_numeric'] = (saf_problem_corr['store_and_fwd_flag'] == 'Y').astype(int)
correlation = saf_problem_corr['saf_numeric'].corr(saf_problem_corr['is_problematic'])

print(f"\n📊 Store-and-Forward vs Problem Correlation: {correlation:.4f}")

saf_trips = df[df['store_and_fwd_flag'] == 'Y']
normal_trips = df[df['store_and_fwd_flag'] == 'N']

saf_problem_rate = saf_trips['is_problematic'].mean() if len(saf_trips) > 0 else 0
normal_problem_rate = normal_trips['is_problematic'].mean() if len(normal_trips) > 0 else 0

print(f"\n📊 Problem Rates by Connection Status:")
print(f"   Store-and-forward trips: {saf_problem_rate*100:.3f}%")
print(f"   Normal trips: {normal_problem_rate*100:.3f}%")
print(f"   Difference: {(saf_problem_rate - normal_problem_rate)*100:+.3f} percentage points")

if saf_problem_rate > normal_problem_rate * 1.2:
    print(f"   ⚠️ ALERT: Connectivity issues linked to {((saf_problem_rate/normal_problem_rate)-1)*100:.1f}% MORE problems")

# 4. PAYMENT TYPE BY VENDOR
print("\n\n4️⃣ PAYMENT TYPE DISTRIBUTION BY VENDOR")
print("-" * 80)

payment_by_vendor = pd.crosstab(df['VendorID'], df['payment_type_label'], normalize='index') * 100
payment_by_vendor = payment_by_vendor.merge(
    vendor_analysis[['VendorID', 'VendorName']],
    left_index=True,
    right_on='VendorID'
).set_index('VendorName')

print("\n📊 Payment Type Distribution (%):")
print(payment_by_vendor.to_string())

print("\n✅ Vendor accountability analysis complete!")

# Store for visualizations
vendor_data = {
    'performance': vendor_analysis,
    'payment_distribution': payment_by_vendor
}

In [ ]:
# ============================================
# SECTION 7: ROOT CAUSE ANALYSIS
# ============================================

print("\n🔍 ROOT CAUSE ANALYSIS")
print("=" * 80)

print("\n1️⃣ COMPARING PROBLEM TRIPS vs NORMAL TRIPS")
print("-" * 80)

# Separate trip types
normal_trips = df[df['payment_type'].isin([1, 2])]  # Credit card or Cash
dispute_trips = df[df['is_dispute'] == 1]
no_charge_trips = df[df['is_no_charge'] == 1]
voided_trips = df[df['is_voided'] == 1]

# Define metrics to compare
metrics = [
    'trip_distance',
    'trip_duration_minutes',
    'fare_amount',
    'fare_per_mile',
    'fare_per_minute',
    'speed_mph',
    'tip_amount',
    'passenger_count',
    'tolls_amount',
    'is_airport',
    'is_rush_hour',
    'is_weekend'
]

# Calculate averages for each group
comparison_data = {
    'Metric': [],
    'Normal': [],
    'Dispute': [],
    'No Charge': [],
    'Voided': []
}

for metric in metrics:
    if metric in df.columns:
        comparison_data['Metric'].append(metric)
        comparison_data['Normal'].append(normal_trips[metric].mean())
        comparison_data['Dispute'].append(dispute_trips[metric].mean() if len(dispute_trips) > 0 else 0)
        comparison_data['No Charge'].append(no_charge_trips[metric].mean() if len(no_charge_trips) > 0 else 0)
        comparison_data['Voided'].append(voided_trips[metric].mean() if len(voided_trips) > 0 else 0)

comparison_df = pd.DataFrame(comparison_data)

print("\n📊 TRIP CHARACTERISTICS COMPARISON:")
print(comparison_df.to_string(index=False))

# 2. STATISTICAL TESTING
print("\n\n2️⃣ STATISTICAL SIGNIFICANCE TESTING (T-Tests)")
print("-" * 80)

from scipy.stats import ttest_ind

significant_differences = []

for metric in ['trip_distance', 'trip_duration_minutes', 'fare_amount', 'fare_per_mile', 'speed_mph']:
    if metric in df.columns:
        normal_values = normal_trips[metric].dropna()
        dispute_values = dispute_trips[metric].dropna()
        
        if len(dispute_values) > 30:  # Minimum sample size
            t_stat, p_value = ttest_ind(normal_values, dispute_values)
            
            normal_mean = normal_values.mean()
            dispute_mean = dispute_values.mean()
            percent_diff = ((dispute_mean - normal_mean) / normal_mean) * 100
            
            print(f"\n📊 {metric}:")
            print(f"   Normal mean: {normal_mean:.2f}")
            print(f"   Dispute mean: {dispute_mean:.2f}")
            print(f"   Difference: {percent_diff:+.1f}%")
            print(f"   P-value: {p_value:.6f}")
            
            if p_value < 0.05:
                print(f"   ✅ SIGNIFICANT difference (p < 0.05)")
                significant_differences.append({
                    'metric': metric,
                    'percent_diff': percent_diff,
                    'p_value': p_value
                })
            else:
                print(f"   → Not significant (p >= 0.05)")

# 3. KEY INSIGHTS
print("\n\n3️⃣ KEY ROOT CAUSE INSIGHTS")
print("-" * 80)

if significant_differences:
    print("\n🎯 STATISTICALLY SIGNIFICANT DIFFERENCES:")
    for diff in sorted(significant_differences, key=lambda x: abs(x['percent_diff']), reverse=True):
        print(f"   • {diff['metric']}: {diff['percent_diff']:+.1f}% difference")

# 4. DISPUTE TRIGGERS ANALYSIS
print("\n\n4️⃣ COMMON DISPUTE TRIGGERS")
print("-" * 80)

dispute_triggers = []

# High fare disputes
high_fare_disputes = dispute_trips[dispute_trips['fare_amount'] > dispute_trips['fare_amount'].quantile(0.75)]
if len(high_fare_disputes) > 0:
    pct = (len(high_fare_disputes) / len(dispute_trips)) * 100
    dispute_triggers.append(f"High fares (>{dispute_trips['fare_amount'].quantile(0.75):.0f}): {pct:.1f}% of disputes")

# Long distance disputes
long_distance_disputes = dispute_trips[dispute_trips['trip_distance'] > 20]
if len(long_distance_disputes) > 0:
    pct = (len(long_distance_disputes) / len(dispute_trips)) * 100
    dispute_triggers.append(f"Long trips (>20 miles): {pct:.1f}% of disputes")

# Long duration disputes
long_duration_disputes = dispute_trips[dispute_trips['trip_duration_minutes'] > 60]
if len(long_duration_disputes) > 0:
    pct = (len(long_duration_disputes) / len(dispute_trips)) * 100
    dispute_triggers.append(f"Long duration (>60 min): {pct:.1f}% of disputes")

# Airport disputes
airport_disputes = dispute_trips[dispute_trips['is_airport'] == 1]
if len(airport_disputes) > 0:
    pct = (len(airport_disputes) / len(dispute_trips)) * 100
    dispute_triggers.append(f"Airport trips: {pct:.1f}% of disputes")

# Rush hour disputes
rush_disputes = dispute_trips[dispute_trips['is_rush_hour'] == 1]
if len(rush_disputes) > 0:
    pct = (len(rush_disputes) / len(dispute_trips)) * 100
    dispute_triggers.append(f"Rush hour: {pct:.1f}% of disputes")

print("\n🎯 TOP DISPUTE TRIGGERS:")
for trigger in dispute_triggers:
    print(f"   • {trigger}")

print("\n✅ Root cause analysis complete!")

# Store for reporting
root_cause_data = {
    'comparison': comparison_df,
    'significant_differences': significant_differences,
    'dispute_triggers': dispute_triggers
}

In [ ]:
# ============================================
# SECTION 8: FRAUD DETECTION SCORING
# ============================================

print("\n🚨 FRAUD DETECTION SCORING SYSTEM")
print("=" * 80)

# 1. BUILD RULE-BASED SCORING FUNCTION
print("\n1️⃣ BUILDING RULE-BASED FRAUD RISK SCORING")
print("-" * 80)

def calculate_fraud_score(row):
    """
    Calculate fraud risk score based on multiple rule-based criteria
    Returns: integer score (0-25+)
    """
    score = 0
    
    # Fare per mile anomalies
    if pd.notna(row['fare_per_mile']):
        if row['fare_per_mile'] < 2 or row['fare_per_mile'] > 20:
            score += 3
    
    # Suspicious distance-fare combinations
    if row['trip_distance'] > 50 and row['fare_amount'] < 100:
        score += 5
    if row['trip_distance'] < 0.5 and row['fare_amount'] > 50:
        score += 4
    
    # Passenger count anomaly
    if row['passenger_count'] > 6:
        score += 2
    
    # Tip anomaly (for credit card only)
    if row['payment_type'] == 1 and row['tip_amount'] > row['fare_amount']:
        score += 4
    
    # Speed anomaly
    if pd.notna(row['speed_mph']):
        if row['trip_duration_minutes'] < 2 and row['trip_distance'] > 5:
            score += 5
        if row['speed_mph'] > 80:
            score += 3
    
    # Problem payment types
    if row['payment_type'] in [3, 4, 6]:
        score += 3
    
    # Zero fare but positive total
    if row['fare_amount'] == 0 and row['total_amount'] > 0:
        score += 4
    
    # Extremely long trips
    if row['trip_duration_minutes'] > 180:
        score += 3
    
    # Extreme short trips with high fare
    if row['trip_duration_minutes'] < 1 and row['fare_amount'] > 20:
        score += 5
    
    return score

print("   ✓ Fraud scoring function defined")

# 2. CALCULATE FRAUD SCORES
print("\n2️⃣ CALCULATING FRAUD RISK SCORES FOR ALL TRIPS")
print("-" * 80)

# Apply fraud scoring (sample first to test if memory allows full dataset)
print("   Calculating scores... (this may take a moment)")

# Calculate scores
df['fraud_risk_score'] = df.apply(calculate_fraud_score, axis=1)

print(f"   ✅ Fraud scores calculated for {len(df):,} trips")

# 3. CLASSIFY RISK LEVELS
print("\n3️⃣ CLASSIFYING TRIPS BY RISK LEVEL")
print("-" * 80)

def classify_risk(score):
    if score <= 2:
        return 'Low Risk'
    elif score <= 5:
        return 'Medium Risk'
    elif score <= 9:
        return 'High Risk'
    else:
        return 'Critical Risk'

df['risk_category'] = df['fraud_risk_score'].apply(classify_risk)

# Risk distribution
risk_distribution = df['risk_category'].value_counts().sort_index()
print("\n📊 FRAUD RISK DISTRIBUTION:")
for category, count in risk_distribution.items():
    pct = (count / len(df)) * 100
    print(f"   {category}: {count:,} trips ({pct:.2f}%)")

# 4. FINANCIAL IMPACT BY RISK CATEGORY
print("\n\n4️⃣ FINANCIAL IMPACT BY RISK CATEGORY")
print("-" * 80)

risk_financial = df.groupby('risk_category').agg({
    'total_amount': ['sum', 'mean', 'count'],
    'is_problematic': 'mean'
}).reset_index()

risk_financial.columns = ['Risk Category', 'Total Revenue', 'Avg Fare', 'Trip Count', 'Problem Rate']

print("\n📊 REVENUE AT RISK:")
print(risk_financial.to_string(index=False))

# Calculate high-risk revenue
high_risk_revenue = df[df['risk_category'].isin(['High Risk', 'Critical Risk'])]['total_amount'].sum()
total_revenue = df['total_amount'].sum()

print(f"\n⚠️ HIGH & CRITICAL RISK TRIPS:")
print(f"   Total revenue: ${high_risk_revenue:,.2f}")
print(f"   Percentage of total: {(high_risk_revenue/total_revenue)*100:.2f}%")

# 5. TOP HIGH-RISK EXAMPLES
print("\n\n5️⃣ EXAMPLES OF HIGH-RISK TRIPS")
print("-" * 80)

high_risk_examples = df[df['risk_category'] == 'Critical Risk'].nlargest(10, 'fraud_risk_score')

print("\n📊 TOP 10 CRITICAL RISK TRIPS:")
display_cols = ['fraud_risk_score', 'trip_distance', 'trip_duration_minutes', 
                'fare_amount', 'total_amount', 'payment_type_label', 
                'fare_per_mile', 'speed_mph']
print(high_risk_examples[display_cols].to_string(index=False))

# 6. SCORE DISTRIBUTION STATISTICS
print("\n\n6️⃣ FRAUD SCORE STATISTICS")
print("-" * 80)

print(f"\n📊 Score Distribution:")
print(f"   Mean: {df['fraud_risk_score'].mean():.2f}")
print(f"   Median: {df['fraud_risk_score'].median():.0f}")
print(f"   Std Dev: {df['fraud_risk_score'].std():.2f}")
print(f"   Max: {df['fraud_risk_score'].max():.0f}")
print(f"   75th percentile: {df['fraud_risk_score'].quantile(0.75):.0f}")
print(f"   95th percentile: {df['fraud_risk_score'].quantile(0.95):.0f}")
print(f"   99th percentile: {df['fraud_risk_score'].quantile(0.99):.0f}")

print("\n✅ Fraud detection scoring complete!")

# Store for reporting
fraud_data = {
    'risk_distribution': risk_distribution,
    'risk_financial': risk_financial,
    'high_risk_examples': high_risk_examples
}

In [ ]:
# Quick diagnostic cell
try:
    print(f"✅ df exists with {len(df):,} records")
    print(f"✅ Columns available: {len(df.columns)}")
except NameError:
    print("❌ df not defined - run cells 1-8 first")